In [7]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats.distributions import chi2

In [25]:
model_order = [
    "Transformer",
    "LSTM",
    "RNN",
    "5gram",
    "4gram",
    "3gram",
    "Transformer-action_td",
    "Transformer-action_lc-as",
    "LSTM-action_td",
    "LSTM-action_lc-as",
    "RNN-action_td",
    "RNN-action_lc-as",
    "5gram_actions_td",
    "5gram_actions_lc-as",
    "4gram_actions_td",
    "4gram_actions_lc-as",
    "3gram_actions_td",
    "3gram_actions_lc-as",
    "RNNG_top_down",
    "RNNG_in_order",
    "SRNNG_top_down",
    "SRNNG_in_order",
    "td",
    "lc-as",
    "llama2-7b",
]

model2name = {
    "random": "random & &",
    "Transformer": "Transformer & &",
    "LSTM": "LSTM & \\textcolor{gray}{\checkmark} &",
    "RNN": "SRN  & \checkmark &",
    "3gram": "Word 3-gram & \checkmark &",
    "4gram": "Word 4-gram & \checkmark &",
    "5gram": "Word 5-gram & \checkmark &",
    "Transformer-action_td": "Trans. PLM & & TD",
    "Transformer-action_lc-as": "Trans. PLM & & LC",
    "LSTM-action_td": "LSTM PLM & \\textcolor{gray}{\checkmark} & TD",
    "LSTM-action_lc-as": "LSTM PLM & \\textcolor{gray}{\checkmark} & LC",
    "RNN-action_td": "SRN PLM & \checkmark & TD",
    "RNN-action_lc-as": "SRN PLM & \checkmark & LC",
    "5gram_actions_td": "5-gram PLM & \checkmark & TD",
    "4gram_actions_td": "4-gram PLM & \checkmark & TD",
    "3gram_actions_td": "3-gram PLM & \checkmark & TD",
    "5gram_actions_lc-as": "5-gram PLM & \checkmark & LC",
    "4gram_actions_lc-as": "4-gram PLM & \checkmark & LC",
    "3gram_actions_lc-as": "3-gram PLM & \checkmark & LC",
    "RNNG_top_down": "RNNG & & TD",
    "SRNNG_top_down": "SRNNG & \checkmark & TD",
    "RNNG_in_order": "RNNG & & LC",
    "SRNNG_in_order": "SRNNG & \checkmark & LC",
    "llama2-7b": "LLaMA2 (7B) & & ",
    "td": "Stack depth & & TD",
    "lc-as": "Stack depth & & LC",
}

metrics = ["model", "correl",  "micro_correl", "consistency", "left_pref"]
metrics_detailed = ["model", "correl",  "micro_correl", "consistency", "left_pref"]
ks = ["0.5", "1.0", "2.0", "3.0", "log"]
k2label = {"0.5": "PPL$^{0.5}$", "1.0": "PPL", "2.0": "PPL$^2$", "3.0": "PPL$^3$", "log": "log(PPL)"}

In [30]:
df = pd.read_csv("../work/results/regression/results_20230908.csv")
df_stack = pd.read_csv("../work/results/regression/results_20231217_stack_depth.csv")
df = pd.concat([df, df_stack], axis=0)

for score, std, top3 in zip(df[df["k"]=="1.0"][metrics].groupby("model").mean().reindex(model_order).iterrows(), df[df["k"]=="1.0"][metrics].groupby("model").std().reindex(model_order).iterrows(), df[(df["k"]=="1.0") & (df["fold"]==0)][["model", "top3_langs"]].set_index("model").loc[model_order]["top3_langs"].apply(lambda x: ", ".join(["\\texttt{" + l.replace("'", "").strip("[] ").replace("1", "R").replace("0", "L").replace("X", "") + "}" for l in x.split(",")]))):
    model = score[0]
    if model in model2name:
        model = model2name[model]
    print(f"{model} & {score[1]['correl']*100:.1f} $\pm$ {std[1]['correl']*100:.1f} & {score[1]['micro_correl']*100:.1f} $\pm$ {std[1]['micro_correl']*100:.1f} & {score[1]['left_pref']*100:.1f} $\pm$ {std[1]['left_pref']*100:.1f} & {top3} \\\\")

Transformer & & & 12.1 $\pm$ 4.3 & 16.7 $\pm$ 6.4 & 23.8 $\pm$ 6.1 & \texttt{LLLLLL}, \texttt{LLRLLL}, \texttt{RLRLLL} \\
LSTM & \textcolor{gray}{\checkmark} & & 10.7 $\pm$ 14.7 & 26.9 $\pm$ 7.4 & -1.2 $\pm$ 11.3 & \texttt{RLRLLL}, \texttt{RLLLLL}, \texttt{RRRRRR} \\
SRN  & \checkmark & & 16.3 $\pm$ 9.4 & 38.3 $\pm$ 3.0 & -3.6 $\pm$ 10.4 & \texttt{RLLLLL}, \texttt{RLRLLL}, \texttt{RLRRLL} \\
Word 5-gram & \checkmark & & 5.4 $\pm$ 1.0 & 17.0 $\pm$ 1.7 & -5.8 $\pm$ 1.6 & \texttt{RRLRRR}, \texttt{RRRRRR}, \texttt{LRLRRR} \\
Word 4-gram & \checkmark & & 6.5 $\pm$ 1.0 & 16.5 $\pm$ 1.6 & -6.4 $\pm$ 1.6 & \texttt{RRLRRR}, \texttt{RRRRRR}, \texttt{LRLRRR} \\
Word 3-gram & \checkmark & & 8.8 $\pm$ 0.7 & 17.5 $\pm$ 1.0 & -6.1 $\pm$ 1.0 & \texttt{RRRRRR}, \texttt{RRLRRR}, \texttt{LRLRRR} \\
Trans. PLM & & TD & 30.4 $\pm$ 5.7 & 29.5 $\pm$ 9.1 & 35.9 $\pm$ 4.7 & \texttt{LLRRLL}, \texttt{LLRRRL}, \texttt{LLRLLL} \\
Trans. PLM & & LC & 30.3 $\pm$ 2.1 & 42.3 $\pm$ 1.1 & 35.2 $\pm$ 2.3 & \texttt{LLLLLL

## k (linking function)

In [31]:
for k in ks:
    print("\multicolumn{9}{c}{" + k2label[k] + "} \\\\")
    for score, std, top3 in zip(df[df["k"]==k][metrics].groupby("model").mean().reindex(model_order).iterrows(), df[df["k"]=="1.0"][metrics].groupby("model").std().reindex(model_order).iterrows(), df[(df["k"]=="1.0") & (df["fold"]==0)][["model", "top3_langs"]].set_index("model").loc[model_order]["top3_langs"].apply(lambda x: ", ".join(["\\texttt{" + l.replace("'", "").strip("[] ").replace("1", "R").replace("0", "L").replace("X", "") + "}" for l in x.split(",")]))):
        model = score[0]
        if model in model2name:
            model = model2name[model]
        print(f"{model} & {score[1]['correl']*100:.1f} $\pm$ {std[1]['correl']*100:.1f} & {score[1]['micro_correl']*100:.1f} $\pm$ {std[1]['micro_correl']*100:.1f} & {score[1]['left_pref']*100:.1f} $\pm$ {std[1]['left_pref']*100:.1f} & {top3} \\\\")

\multicolumn{9}{c}{PPL$^{0.5}$} \\
Transformer & & & 12.4 $\pm$ 4.3 & 16.7 $\pm$ 6.4 & 24.1 $\pm$ 6.1 & \texttt{LLLLLL}, \texttt{LLRLLL}, \texttt{RLRLLL} \\
LSTM & \textcolor{gray}{\checkmark} & & 11.2 $\pm$ 14.7 & 27.0 $\pm$ 7.4 & -0.8 $\pm$ 11.3 & \texttt{RLRLLL}, \texttt{RLLLLL}, \texttt{RRRRRR} \\
SRN  & \checkmark & & 16.5 $\pm$ 9.4 & 38.4 $\pm$ 3.0 & -3.5 $\pm$ 10.4 & \texttt{RLLLLL}, \texttt{RLRLLL}, \texttt{RLRRLL} \\
Word 5-gram & \checkmark & & 5.5 $\pm$ 1.0 & 17.1 $\pm$ 1.7 & -6.0 $\pm$ 1.6 & \texttt{RRLRRR}, \texttt{RRRRRR}, \texttt{LRLRRR} \\
Word 4-gram & \checkmark & & 6.6 $\pm$ 1.0 & 16.7 $\pm$ 1.6 & -6.7 $\pm$ 1.6 & \texttt{RRLRRR}, \texttt{RRRRRR}, \texttt{LRLRRR} \\
Word 3-gram & \checkmark & & 8.9 $\pm$ 0.7 & 17.7 $\pm$ 1.0 & -6.3 $\pm$ 1.0 & \texttt{RRRRRR}, \texttt{RRLRRR}, \texttt{LRLRRR} \\
Trans. PLM & & TD & 30.5 $\pm$ 5.7 & 29.4 $\pm$ 9.1 & 36.2 $\pm$ 4.7 & \texttt{LLRRLL}, \texttt{LLRRRL}, \texttt{LLRLLL} \\
Trans. PLM & & LC & 30.2 $\pm$ 2.1 & 42.3 $\pm$ 1.

## parseability

In [32]:
df = pd.read_csv("../work/results/regression/results_20230927_parseability.csv")
model_rnngs = ["RNNG_top_down_beam", "SRNNG_top_down_beam", "RNNG_in_order_beam", "SRNNG_in_order_beam"]

for score, std, top3 in zip(df[(df["k"]=="1.0")][metrics].groupby("model").mean().reindex(model_rnngs).iterrows(), df[df["k"]=="1.0"][metrics].groupby("model").std().reindex(model_rnngs).iterrows(), df[(df["k"]=="1.0") & (df["fold"]==0)][["model", "top3_langs"]].set_index("model").loc[model_rnngs]["top3_langs"].apply(lambda x: ", ".join(["\\texttt{" + l.replace("'", "").strip("[] ").replace("1", "R").replace("0", "L").replace("X", "") + "}" for l in x.split(",")]))):
    model = score[0]
    if model in model2name:
        model = model2name[model]
    print(f"{model} & {score[1]['correl']*100:.1f} $\pm$ {std[1]['correl']*100:.1f} & {score[1]['micro_correl']*100:.1f} $\pm$ {std[1]['micro_correl']*100:.1f} & {score[1]['left_pref']*100:.1f} $\pm$ {std[1]['left_pref']*100:.1f} & {top3} \\\\")

RNNG_top_down_beam & -2.5 $\pm$ 1.1 & 26.5 $\pm$ 2.5 & 24.9 $\pm$ 2.4 & \texttt{RLRLLL}, \texttt{RLLLLL}, \texttt{RLLRLL} \\
SRNNG_top_down_beam & -6.5 $\pm$ 2.2 & 14.0 $\pm$ 1.5 & 22.6 $\pm$ 7.1 & \texttt{RLRLLL}, \texttt{RLRRLL}, \texttt{RLLRLL} \\
RNNG_in_order_beam & 10.1 $\pm$ 6.5 & 17.1 $\pm$ 3.0 & -19.7 $\pm$ 10.0 & \texttt{LRRRLL}, \texttt{RRRRRR}, \texttt{RRRLRR} \\
SRNNG_in_order_beam & 8.2 $\pm$ 3.1 & -12.5 $\pm$ 1.8 & -20.6 $\pm$ 8.6 & \texttt{RRRRRR}, \texttt{RRRLRR}, \texttt{RRLRRR} \\


## Parseability and predictability test

In [33]:
df = pd.read_csv("../work/results/regression/results_20230927_parseability_predictability.csv")
for i in df["delta loglik"]:
    print(chi2.sf(i*2, 1))

0.3677686386527389
0.2930154839690427
0.1642837553774918
0.36302292764955457
0.8254732376117565
0.1570928885548097
0.6178976030214567
0.49198466523333784
0.16364639083178073
0.7365362655898203
0.7720757054796396
0.5330888155828633
0.7320142456921206
0.8670084955117991
0.37790288524291127
0.400506280333491
0.5780045715161701
0.4330922122971129
0.27297968225974256
0.5570287598275236


## Beam

In [36]:
df = pd.read_csv("../work/results/regression/results_20230908.csv")

model_rnngs = ["RNNG_top_down", "SRNNG_top_down", "RNNG_in_order", "SRNNG_in_order",
               "RNNG_top_down_beam", "SRNNG_top_down_beam", "RNNG_in_order_beam", "SRNNG_in_order_beam"]

rnng2name = {
    "RNNG_top_down": "RNNG & TD & &",
    "SRNNG_top_down": "SRNNG & TD & \checkmark &",
    "RNNG_in_order": "RNNG & LC & &",
    "SRNNG_in_order": "SRNNG & LC & \checkmark &",
    "RNNG_top_down_beam": "RNNG & TD & & \checkmark",
    "SRNNG_top_down_beam": "SRNNG & TD & \checkmark & \checkmark",
    "RNNG_in_order_beam": "RNNG & LC & & \checkmark",
    "SRNNG_in_order_beam": "SRNNG & LC & \checkmark & \checkmark",
}

for score, std, top3 in zip(df[df["k"]=="1.0"][metrics].groupby("model").mean().reindex(model_rnngs).iterrows(), df[df["k"]=="1.0"][metrics].groupby("model").std().reindex(model_rnngs).iterrows(), df[df["fold"]==0][["model", "top3_langs"]].set_index("model").loc[model_rnngs]["top3_langs"].apply(lambda x: ", ".join(["\\texttt{" + l.replace("'", "").strip("[] ").replace("1", "R").replace("0", "L").replace("X", "") + "}" for l in x.split(",")]))):
    model = score[0]
    if model in rnng2name:
        model = rnng2name[model]
    print(f"{model} & {score[1]['correl']*100:.1f} $\pm$ {std[1]['correl']*100:.1f} & {score[1]['micro_correl']*100:.1f} $\pm$ {std[1]['micro_correl']*100:.1f}  & {score[1]['left_pref']*100:.1f} $\pm$ {std[1]['left_pref']*100:.1f} & {top3} \\\\")

RNNG & TD & & & -22.6 $\pm$ 4.7 & 6.0 $\pm$ 14.5  & 14.5 $\pm$ 10.8 & \texttt{RLLRLL}, \texttt{RRRRLL}, \texttt{RRRLLL} \\
SRNNG & TD & \checkmark & & 2.0 $\pm$ 9.3 & 10.7 $\pm$ 7.8  & 10.2 $\pm$ 7.0 & \texttt{RLLRLL}, \texttt{RRRRLL}, \texttt{RRRLLL} \\
RNNG & LC & & & -17.6 $\pm$ 6.4 & 25.1 $\pm$ 13.4  & -21.2 $\pm$ 2.0 & \texttt{RLLRLL}, \texttt{RRRRLL}, \texttt{RRRLLL} \\
SRNNG & LC & \checkmark & & 19.1 $\pm$ 9.5 & 23.7 $\pm$ 12.9  & -40.6 $\pm$ 8.5 & \texttt{RLLRLL}, \texttt{RRRRLL}, \texttt{RRRLLL} \\
RNNG & TD & & \checkmark & 9.4 $\pm$ 3.5 & -31.5 $\pm$ 11.6  & -30.1 $\pm$ 5.7 & \texttt{RLLRLL}, \texttt{RRRRLL}, \texttt{RRRLLL} \\
SRNNG & TD & \checkmark & \checkmark & 14.6 $\pm$ 8.7 & -2.8 $\pm$ 5.9  & -21.5 $\pm$ 8.9 & \texttt{RLLRRR}, \texttt{RLRRRR}, \texttt{RLLRRL} \\
RNNG & LC & & \checkmark & -23.4 $\pm$ 7.0 & 26.5 $\pm$ 13.9  & -26.2 $\pm$ 7.2 & \texttt{RLLRRR}, \texttt{RLRRRR}, \texttt{RLLRRL} \\
SRNNG & LC & \checkmark & \checkmark & 17.2 $\pm$ 8.5 & 18.3 $\pm$ 12.4 

## Statistical tests

In [37]:
model2architecture = {
    "Transformer": "Transformer",
    "LSTM": "LSTM",
    "RNN": "SRN",
    "3gram": "3-gram",
    "4gram": "4-gram",
    "5gram": "5-gram",
    "Transformer-action_td": "Transformer",
    "Transformer-action_lc-as": "Transformer",
    "LSTM-action_td":  "LSTM",
    "LSTM-action_lc-as": "LSTM",
    "RNN-action_td": "SRN",
    "RNN-action_lc-as": "SRN",
    "5gram_actions_td": "5-gram",
    "4gram_actions_td": "4-gram",
    "3gram_actions_td": "3-gram",
    "5gram_actions_lc-as": "5-gram",
    "4gram_actions_lc-as": "4-gram",
    "3gram_actions_lc-as": "3-gram",
    "RNNG_top_down": "RNNG",
    "SRNNG_top_down": "SRNNG",
    "RNNG_in_order": "RNNG",
    "SRNNG_in_order": "SRNNG",
    "llama2-7b": "LLaMA 2",
    "td": "Stack",
    "lc-as": "Stack",
}

model2syntax = {
    "Transformer": False,
    "LSTM": False,
    "RNN": False,
    "5gram": False,
    "4gram": False,
    "3gram": False,
    "Transformer-action_td": True,
    "Transformer-action_lc-as":  True,
    "LSTM-action_td": True,
    "LSTM-action_lc-as": True,
    "RNN-action_td": True,
    "RNN-action_lc-as": True,
    "5gram_actions_td": True,
    "5gram_actions_lc-as": True,
    "4gram_actions_td": True,
    "4gram_actions_lc-as": True,
    "3gram_actions_td": True,
    "3gram_actions_lc-as": True,
    "RNNG_top_down": True,
    "SRNNG_top_down": True,
    "RNNG_in_order": True,
    "SRNNG_in_order": True,
}

model2traversal = {
    "Transformer": "No",
    "LSTM": "No",
    "RNN": "No",
    "3gram": "No",
    "4gram": "No",
    "5gram": "No",
    "Transformer-action_td": "TD",
    "Transformer-action_lc-as": "LC",
    "LSTM-action_td": "TD",
    "LSTM-action_lc-as": "LC",
    "RNN-action_td": "TD",
    "RNN-action_lc-as": "LC",
    "5gram_actions_td": "TD",
    "4gram_actions_td": "TD",
    "3gram_actions_td": "TD",
    "5gram_actions_lc-as": "LC",
    "4gram_actions_lc-as": "LC",
    "3gram_actions_lc-as": "LC",
    "RNNG_top_down": "TD",
    "SRNNG_top_down": "TD",
    "RNNG_in_order": "LC",
    "SRNNG_in_order": "LC",
    "td": "TD",
    "lc-as": "LC",
}

model2memory = {
    "Transformer": 1,
    "LSTM": 2,
    "RNN": 3,
    "3gram": 3,
    "4gram": 2,
    "5gram": 1,
    "Transformer-action_td": 1,
    "Transformer-action_lc-as": 1,
    "LSTM-action_td": 2,
    "LSTM-action_lc-as": 2,
    "RNN-action_td": 3,
    "RNN-action_lc-as": 3,
    "5gram_actions_td": 1,
    "4gram_actions_td": 2,
    "3gram_actions_td": 3,
    "5gram_actions_lc-as": 1,
    "4gram_actions_lc-as": 2,
    "3gram_actions_lc-as": 3,
    "RNNG_top_down": 1,
    "SRNNG_top_down":2,
    "RNNG_in_order": 1,
    "SRNNG_in_order": 2,
}

model2neural = {
    "Transformer": 1,
    "LSTM": 1,
    "RNN": 1,
    "3gram": 0,
    "4gram": 0,
    "5gram": 0,
    "Transformer-action_td": 1,
    "Transformer-action_lc-as": 1,
    "LSTM-action_td": 1,
    "LSTM-action_lc-as": 1,
    "RNN-action_td": 1,
    "RNN-action_lc-as": 1,
    "5gram_actions_td": 0,
    "4gram_actions_td": 0,
    "3gram_actions_td": 0,
    "5gram_actions_lc-as": 0,
    "4gram_actions_lc-as": 0,
    "3gram_actions_lc-as": 0,
    "RNNG_top_down": 1,
    "SRNNG_top_down":1,
    "RNNG_in_order": 1,
    "SRNNG_in_order": 1,
}

model2class = {
    "Transformer": "NLM",
    "LSTM": "NLM",
    "RNN": "NLM",
    "3gram": "CLM",
    "4gram": "CLM",
    "5gram": "CLM",
    "Transformer-action_td": "NLM",
    "Transformer-action_lc-as": "NLM",
    "LSTM-action_td": "NLM",
    "LSTM-action_lc-as": "NLM",
    "RNN-action_td": "NLM",
    "RNN-action_lc-as": "NLM",
    "5gram_actions_td": "CLM",
    "4gram_actions_td": "CLM",
    "3gram_actions_td": "CLM",
    "5gram_actions_lc-as": "CLM",
    "4gram_actions_lc-as": "CLM",
    "3gram_actions_lc-as": "CLM",
    "RNNG_top_down": "RNNG",
    "SRNNG_top_down":  "RNNG",
    "RNNG_in_order": "RNNG",
    "SRNNG_in_order":  "RNNG",
}


In [38]:
df = pd.read_csv("../work/results/regression/results_20230908.csv")
df = df[df["k"]=="1.0"]
df_stack = pd.read_csv("../work/results/regression/results_20231217_stack_depth.csv")
df_stack = df_stack[df_stack["k"]=="1.0"]
df = pd.concat([df, df_stack])

df["architecture"] = df["model"].map(model2architecture)
df["syntax"] = df["model"].map(model2syntax)
df["traversal"] = df["model"].map(model2traversal)
df["memory"] = df["model"].map(model2memory)
df["neural"] = df["model"].map(model2neural)
df["class"] = df["model"].map(model2class)
df_sub = df[df["model"].isin(list(model2syntax.keys()))]

for row in df_sub[["model", "class", "memory", "syntax", "traversal"]].drop_duplicates().set_index("model").reindex(model_order).dropna().iterrows():
    print(f"{model2name[row[0]].split('&')[0]} & {row[1]['class']} & {int(row[1]['memory'])} & {row[1]['syntax']} & {row[1]['traversal'].replace('No', '-')} \\\\")

Transformer  & NLM & 1 & False & - \\
LSTM  & NLM & 2 & False & - \\
SRN   & NLM & 3 & False & - \\
Word 5-gram  & CLM & 1 & False & - \\
Word 4-gram  & CLM & 2 & False & - \\
Word 3-gram  & CLM & 3 & False & - \\
Trans. PLM  & NLM & 1 & True & TD \\
Trans. PLM  & NLM & 1 & True & LC \\
LSTM PLM  & NLM & 2 & True & TD \\
LSTM PLM  & NLM & 2 & True & LC \\
SRN PLM  & NLM & 3 & True & TD \\
SRN PLM  & NLM & 3 & True & LC \\
5-gram PLM  & CLM & 1 & True & TD \\
5-gram PLM  & CLM & 1 & True & LC \\
4-gram PLM  & CLM & 2 & True & TD \\
4-gram PLM  & CLM & 2 & True & LC \\
3-gram PLM  & CLM & 3 & True & TD \\
3-gram PLM  & CLM & 3 & True & LC \\
RNNG  & RNNG & 1 & True & TD \\
RNNG  & RNNG & 1 & True & LC \\
SRNNG  & RNNG & 2 & True & TD \\
SRNNG  & RNNG & 2 & True & LC \\


In [39]:
# Global correlations, all models

traversal_dumies = pd.get_dummies(df_sub["traversal"], prefix="traversal")
class_dumies = pd.get_dummies(df_sub["class"], prefix="class")
X = pd.concat([class_dumies, df_sub, traversal_dumies], axis=1)
model = ols('correl ~ class_NLM + class_CLM + class_RNNG + syntax + memory + traversal_LC', data=X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 correl   R-squared:                       0.378
Model:                            OLS   Adj. R-squared:                  0.349
Method:                 Least Squares   F-statistic:                     12.67
Date:                Wed, 02 Oct 2024   Prob (F-statistic):           1.30e-09
Time:                        20:24:37   Log-Likelihood:                 81.940
No. Observations:                 110   AIC:                            -151.9
Df Residuals:                     104   BIC:                            -135.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0286 

In [40]:
# Global correlations, without neural PLMs

X = X[~((X["class_NLM"]) & (X["syntax"]))]
model = ols('correl ~ class_NLM + class_CLM + class_RNNG + syntax + memory + traversal_LC', data=X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 correl   R-squared:                       0.548
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     17.95
Date:                Wed, 02 Oct 2024   Prob (F-statistic):           1.29e-11
Time:                        20:24:41   Log-Likelihood:                 74.010
No. Observations:                  80   AIC:                            -136.0
Df Residuals:                      74   BIC:                            -121.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.0643 

In [41]:
# Local correlations, all models

traversal_dumies = pd.get_dummies(df_sub["traversal"], prefix="traversal")
class_dumies = pd.get_dummies(df_sub["class"], prefix="class")
X = pd.concat([class_dumies, df_sub, traversal_dumies], axis=1)

model = ols('micro_correl ~ class_NLM + class_CLM + class_RNNG + syntax + memory + traversal_LC', data=X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           micro_correl   R-squared:                       0.525
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                     23.01
Date:                Wed, 02 Oct 2024   Prob (F-statistic):           1.71e-15
Time:                        20:24:44   Log-Likelihood:                 89.964
No. Observations:                 110   AIC:                            -167.9
Df Residuals:                     104   BIC:                            -151.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.1374 

In [27]:
# Local correlations, without neural PLMs

X = X[~((X["class_NLM"]) & (X["syntax"]))]
model = ols('micro_correl ~ class_NLM + class_CLM + class_RNNG + syntax + memory + traversal_LC', data=X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           micro_correl   R-squared:                       0.739
Model:                            OLS   Adj. R-squared:                  0.721
Method:                 Least Squares   F-statistic:                     41.90
Date:                Sat, 13 Jul 2024   Prob (F-statistic):           2.99e-20
Time:                        16:25:40   Log-Likelihood:                 85.216
No. Observations:                  80   AIC:                            -158.4
Df Residuals:                      74   BIC:                            -144.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0634 